# Nifti Read Example

The purpose of this notebook is to illustrate reading Nifti files and iterating over patches of the volumes loaded from them.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/modules/nifti_read_example.ipynb)

## Setup environment

In [1]:
!python -c "import monai" || pip install -q monai[nibabel]

## Setup imports

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import glob
import os
import shutil
import tempfile

import nibabel as nib
import numpy as np
import torch

from monai.config import print_config
from monai.data import ArrayDataset, GridPatchDataset, create_test_image_3d
from monai.transforms import (
    AddChannel,
    Compose,
    LoadImage,
    RandSpatialCrop,
    ScaleIntensity,
    ToTensor,
)
from monai.utils import first

print_config()

MONAI version: 0.4.0+35.g6adbcde
Numpy version: 1.19.5
Pytorch version: 1.7.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 6adbcdee45c16f18f5b713575af3410437177311

Optional dependencies:
Pytorch Ignite version: 0.4.2
Nibabel version: 3.2.1
scikit-image version: 0.18.1
Pillow version: 7.0.0
Tensorboard version: 2.4.0
gdown version: 3.12.2
TorchVision version: 0.8.2
ITK version: 5.1.2
tqdm version: 4.51.0
lmdb version: 1.0.0
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/data/medical


Create a number of test Nifti files:

In [4]:
for i in range(5):
    im, seg = create_test_image_3d(128, 128, 128)

    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"im{i}.nii.gz"))

    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"seg{i}.nii.gz"))

Create a data loader which yields uniform random patches from loaded Nifti files:

In [5]:
images = sorted(glob.glob(os.path.join(root_dir, "im*.nii.gz")))
segs = sorted(glob.glob(os.path.join(root_dir, "seg*.nii.gz")))

imtrans = Compose(
    [
        LoadImage(image_only=True),
        ScaleIntensity(),
        AddChannel(),
        RandSpatialCrop((64, 64, 64), random_size=False),
        ToTensor(),
    ]
)

segtrans = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        RandSpatialCrop((64, 64, 64), random_size=False),
        ToTensor(),
    ]
)

ds = ArrayDataset(images, imtrans, segs, segtrans)

loader = torch.utils.data.DataLoader(
    ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available()
)
im, seg = first(loader)
print(im.shape, seg.shape)

torch.Size([5, 1, 64, 64, 64]) torch.Size([5, 1, 64, 64, 64])


Alternatively create a data loader which yields patches in regular grid order from loaded images:

In [6]:
imtrans = Compose([LoadImage(image_only=True),
                   ScaleIntensity(), AddChannel(), ToTensor()])

segtrans = Compose([LoadImage(image_only=True), AddChannel(), ToTensor()])

ds = ArrayDataset(images, imtrans, segs, segtrans)
ds = GridPatchDataset(ds, (64, 64, 64))

loader = torch.utils.data.DataLoader(
    ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available()
)
im, seg = first(loader)
print(im.shape, seg.shape)

torch.Size([10, 1, 64, 64, 64]) torch.Size([10, 1, 64, 64, 64])


## Cleanup data directory

Remove directory if a temporary was used.

In [7]:
if directory is None:
    shutil.rmtree(root_dir)